# Data Munging

In [31]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [3]:
raw_train_data = pd.read_csv("sales_train.csv")
category_data = pd.read_csv("items.csv").loc[:,["item_id", "item_category_id"]]
df = raw_train_data.join(category_data.set_index("item_id"), on = "item_id").drop("date", axis = 1)
category_data_dict = {x[0]:x[1] for x in category_data.itertuples(index = False)}
df = df.sample(frac=0.005, replace = True)

# One hot encoding

In [4]:
# not encoding item_id becasue possible unique set size is 22170
# for column in ["shop_id", "item_category_id"]:
#     df = pd.concat([df ,pd.get_dummies(df[column], prefix = column)], axis = 1)
#     df.drop([column], axis = 1, inplace = True)
# df.head()

# Abalation//pending

In [5]:
print(len(set(category_data.item_category_id.unique())))
print(len(set(category_data.item_id.unique())))
print(len(set(raw_train_data.shop_id.unique())))

84
22170
60


In [6]:
# def getSubSampleTrainTestSet(data_set,):
sample_item_set = set(sample_train.item_id)
len(sample_item_set)
def isInTraining(item_id):
    if item_id in sample_item_set:
        return True
    return False
def getCategoryIdForItem(item_id):
#     return category_data.loc[category_data["item_id"] == item_id , "item_id"]
    return category_data_dict[item_id]



NameError: name 'sample_train' is not defined

In [ ]:
% matplotlib inline

In [ ]:
_df = df[["shop_id", "item_cnt_day", "item_price","item_category_id", "item_id"]].loc[(df["item_price"]<1000) 
& (df["item_cnt_day"] < 5) & (df["item_price"] > 0) & (df["item_cnt_day"] > 0)]
# _df = df[['item_price','item_cnt_day','date_block_num','shop_id']].loc[(df['item_price']>0) & (df['item_price']<100000) & (df['item_cnt_day']<50)]

In [ ]:
plt.scatter(_df["shop_id"], _df["item_cnt_day"], s=0.1)

In [ ]:
pd.unique(_df["shop_id"])

In [ ]:
plt.scatter(_df["item_category_id"], _df["item_cnt_day"], s=0.1)

In [ ]:
plt.scatter(_df["item_price"], _df["item_cnt_day"], s=0.1)

In [ ]:
plt.scatter(_df["item_id"], _df["shop_id"], s=0.1)

In [ ]:
plt.scatter(_df["item_id"], _df["item_category_id"], s=0.1)

In [ ]:
plt.scatter(_df["shop_id"], _df["item_category_id"], s=0.1)

In [ ]:
_df.corr()

In [ ]:
plt.scatter(_df["item_price"], _df["item_category_id"], s=0.1)

In [ ]:
# from mpl_toolkits.mplot3d import Axes3D
# figure = plt.figure()
# ax = figure.add_subplot(111, projection="3d")
# ax.scatter(_df["item_price"], _df["item_category_id"], _df["item_id"])
len(pd.unique(df["item_price"].astype(int)))

In [ ]:
len(pd.unique(df["item_category_id"]))

# Model Fitting

In [7]:
raw_train_data = pd.read_csv("sales_train.csv")
category_data = pd.read_csv("items.csv").loc[:,["item_id", "item_category_id"]]
df = raw_train_data.join(category_data.set_index("item_id"), on = "item_id").drop("date", axis = 1)
category_data_dict = {x[0]:x[1] for x in category_data.itertuples(index = False)}
df = df.sample(frac=0.001, replace = True)

for column in ["shop_id", "item_category_id"]:
    df = pd.concat([df ,pd.get_dummies(df[column], prefix = column)], axis = 1)
    df.drop([column], axis = 1, inplace = True)
df.drop(["item_id"], axis = 1, inplace = True)

In [8]:
df_train, df_test = train_test_split(df, test_size=0.2)

In [13]:
lm = linear_model.LinearRegression()
df_train_Y = df_train.filter(regex="item_cnt_day")
df_train_X = df_train.drop("item_cnt_day", axis=1, inplace = False)
df_test_Y = df_train.filter(regex="item_cnt_day")
df_test_X = df_train.drop("item_cnt_day", axis=1, inplace = False)

In [19]:
lm.fit(df_train_X,df_train_Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [20]:
df_test_pred = lm.predict(df_test_X)

In [24]:

df_test_Y.values.shape

(2348, 1)

In [25]:
df_test_pred.shape

(2348, 1)

In [32]:
metrics.mean_absolute_error(df_test_Y.values, df_test_pred)

0.39983463812567244

In [33]:
metrics.mean_squared_error(df_test_Y.values, df_test_pred)

1.631286356293189